# Ho-Lee model

In [1]:
from zero import zeros, forward_rate
import pandas as pd

_bond_data = [
    [100, 0.50, 0, 99.1338],
    [100, 1.00, 0, 97.8925],
    [100, 1.50, 0, 96.1531]
]

# calculate zero coupon bond's yield
_yield = list(map(lambda x: round(x, 4), zeros(_bond_data)))
print("Yield: " + str(_yield))
# calculate forward rate
_forward = list(map(lambda x: round(x, 4), forward_rate(_bond_data, 0.5)))
print("Forward Rate: " + str(_forward))
print()

_market_data = pd.DataFrame(
    {
        "Maturity": [0.5, 1.0, 1.5],
        "Price": [99.1338, 97.8925, 96.1531],
        "Yield": _yield,
        "Forward Rate": _forward
    }
)

mp0 = _market_data.Price[0]
mp1 = _market_data.Price[1]
mp2 = _market_data.Price[2]

print(_market_data)

Yield: [0.0174, 0.0213, 0.0262]
Forward Rate: [0.0174, 0.0252, 0.0359]

   Maturity    Price   Yield  Forward Rate
0       0.5  99.1338  0.0174        0.0174
1       1.0  97.8925  0.0213        0.0252
2       1.5  96.1531  0.0262        0.0359


In [2]:
import binomial as bi
from price import round_model

r00 = 0.0174
theta0 = 0.01
theta1 = 0.00
delta = 0.5
vol = 0.0173

r10 = r00 + theta0*delta + vol*delta**0.5
r11 = r00 + theta0*delta - vol*delta**0.5

print("Interest rate going up: " + str(round(r10*100, 2)) + '%')
print("Interest rate going down: " + str(round(r11*100, 2)) + '%')

irm = bi.model([[r00], [r10, r11]])
round_model(irm, 4)
print("Interest rate model:")
print(irm.data)

Interest rate going up: 3.46%
Interest rate going down: 1.02%
Interest rate model:
[[0.0174], [0.0346, 0.0102]]


In [3]:
from price import bond_model

bpm = round_model(bond_model(100, irm, [0.5], 0.5), 4)
print("Bond price model:")
print(bpm.data)

Bond price model:
[[98.0315], [98.2849, 99.4913], [100, 100, 100, 100]]


In [4]:
from scipy.optimize import fsolve

ir_by_ho_lee = lambda ir, theta, vol, delta: [ir+theta*delta + vol*delta**0.5, ir+theta*delta - vol*delta**0.5]
get_irm = lambda th: bi.model([[r00], ir_by_ho_lee(r00, th, vol, delta)])
get_bpm = lambda th: bond_model(100, get_irm(th), [0.5], 0.5)
price_error = lambda th: (get_bpm(th).value(0, 0)-mp1)**2

_result = fsolve(price_error, theta0)
print("Get result: " + str(_result))

theta0 = _result[0]
print("theta0: " + str(round(theta0*100, 4)) + '%')
print()

irm1 = get_irm(theta0)
print("Interest rate model:")
print(irm1.data)
print()

bpm1 = get_bpm(theta0)
print("Bond price model:")
print(bpm1.data)

Get result: [0.01567581]
theta0: 1.5676%

Interest rate model:
[[0.0174], [0.037470854672668315, 0.013004960043613768]]

Bond price model:
[[97.89250000000004], [98.14389898238744, 99.35186153525606], [100, 100, 100, 100]]


In [5]:
def _get_next_irm(__irm, __th, __vol, __delta):
    next_irm_data = []
    _next_data = []
    for i in __irm.data[-1]:
        for j in ir_by_ho_lee(i, __th, __vol, __delta):
            _next_data.append(j)
    for i in __irm.data:
        next_irm_data.append(i)
    next_irm_data.append(_next_data)
    return bi.model(next_irm_data)

get_bpm = lambda th: bond_model(100, _get_next_irm(irm1, th, vol, delta), [0.5, 0.5], 0.5)
price_error = lambda th: (get_bpm(th).value(0, 0)-mp2)**2

_result = fsolve(price_error, theta1)
print("Get result: " + str(_result))

theta1 = _result[0]
print("theta1: " + str(round(theta1*100, 4)) + '%')
print()

irm2 = round_model(_get_next_irm(irm1, theta1, vol, delta), 4)
print("irm:")
print(irm2.data)
print()

bpm2 = round_model(bond_model(100, irm2, [0.5, 0.5], 0.5),4)
print("bpm:")
print(bpm2.data)

Get result: [0.02153637]
theta1: 2.1536%

irm:
[[0.0174], [0.0375, 0.013], [0.0605, 0.036, 0.036, 0.0115]]

bpm:
[[96.1528], [95.8049, 98.1811], [97.0203, 98.2161, 98.2161, 99.4266], [100, 100, 100, 100, 100, 100, 100, 100]]
